<a href="https://colab.research.google.com/github/ML-HW-SYS/a3-WDaugherty/blob/main/4_gemm_gpu.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# GEMM on GPU

## 1. Set-up 

In [1]:
# Mount google drive 
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
# Make sure your token is stored in a txt file at the location below.
# This way there is no risk that you will push it to your repo
# Never share your token with anyone, it is basically your github password!
with open('/content/gdrive/MyDrive/ece5545/token.txt') as f:
    token = f.readline().strip()
# Use another file to store your github username    
# with open('/content/gdrive/MyDrive/ece5545/git_username.txt') as f:
#     handle = f.readline().strip()

In [22]:
# Clone your github repo
YOUR_TOKEN = token
YOUR_HANDLE = 'WDaugherty'
BRANCH = "main"

%mkdir /content/gdrive/MyDrive/ece5545
%cd /content/gdrive/MyDrive/ece5545
!git clone https://{YOUR_TOKEN}@github.com/ML-HW-SYS/a3-{YOUR_HANDLE}.git
%cd /content/gdrive/MyDrive/ece5545/a3-{YOUR_HANDLE}
!git checkout {BRANCH}
!git pull 
%cd /content/gdrive/MyDrive/ece5545

PROJECT_ROOT = f"/content/gdrive/MyDrive/ece5545/a3-{YOUR_HANDLE}"

mkdir: cannot create directory ‘/content/gdrive/MyDrive/ece5545’: File exists
/content/gdrive/MyDrive/ece5545
fatal: destination path 'a3-WDaugherty' already exists and is not an empty directory.
/content/gdrive/MyDrive/ece5545/a3-WDaugherty
Already on 'main'
Your branch is up to date with 'origin/main'.
remote: Enumerating objects: 7, done.
remote: Counting objects: 100% (7/7), done.
remote: Compressing objects: 100% (2/2), done.
remote: Total 4 (delta 2), reused 4 (delta 2), pack-reused 0
Unpacking objects: 100% (4/4), 413 bytes | 14.00 KiB/s, done.
From https://github.com/ML-HW-SYS/a3-WDaugherty
   79ad084..3089c21  main       -> origin/main
Updating 79ad084..3089c21
Fast-forward
 src/ops.py | 12 ++++++------
 1 file changed, 6 insertions(+), 6 deletions(-)
/content/gdrive/MyDrive/ece5545


In [4]:
# This extension reloads all imports before running each cell
%load_ext autoreload
%autoreload 2

In [5]:
!ls {PROJECT_ROOT}

1-conv1d_cpu.ipynb  2_conv1d_gpu.ipynb	 5-conv2d_dw_gpu.ipynb	src
1_conv1d_cpu.ipynb  3-conv1d_fpga.ipynb  leaderboard_id.txt	tests
2-conv1d_gpu.ipynb  4-gemm_gpu.ipynb	 README.md


## 2. Install TVM

In [6]:
!pip install tlcpack-nightly-cu102 -f https://tlcpack.ai/wheels

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://tlcpack.ai/wheels
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 407.9/407.9 MB 4.0 MB/s eta 0:00:00


## 3. Implement `make_conv1d_gpu_scheduler_func` function in `src.ops`

In that function, you are required to implemented 1D convolution and use TVM to optimize it.
Let $x \in \mathbb{R}^m$ and $y \in \mathbb{R}^n$, then 
$$
\operatorname{conv1d}(x, y)_i = \sum_{j=-\infty}^{\infty} x[j]y[i-j], \forall i \in \{0, 1, \dots, m + n - 1\}
$$

Please use zero padding and unit stride. Please see the numpy convolution function for more detail: [link](https://numpy.org/doc/stable/reference/generated/numpy.convolve.html).

The `make_conv1d_gpu_scheduler_func` takes $m$ and $n$, which are the size of the two 1D input array. 
You should return both the TVM scheduler and the TVM opterator for 
1. Input $x$
2. Input $y$
3. Output $out$

The scheduler should be able to used to build a function with signature $func(x, y, out)$. 
Please see the following cells for usage.

In [23]:
import tvm
import numpy as np
import sys
# Adding assignment 3 to the system path
# Make sure this matches your git directory
sys.path.insert(0, PROJECT_ROOT)
from src.ops import make_gemm_gpu_scheduler

M = 1024
N = 512
K = 2048
dtype = 'float32'
a_np = np.random.rand(M, K).astype(dtype)
w_np = np.random.rand(K, N).astype(dtype)
b_np = np.matmul(a_np, w_np)

s, A, W, B = make_gemm_gpu_scheduler(M, K, N) 
func = tvm.build(s, [A, W, B], "cuda")

dev = tvm.cuda(0)
a = tvm.nd.array(a_np, dev)
w = tvm.nd.array(w_np, dev)
b = tvm.nd.array(np.zeros((M, N), dtype), dev)
func(a, w, b)
evaluator = func.time_evaluator(func.entry_name, dev, number=1, repeat =1)


print("Answer:", b_np)
print("Output:", b)
print(f"1DConv TVM: %f ms" % (evaluator(a, w, b).mean * 1e3))

Answer: [[512.76294 516.04083 504.88028 ... 512.8178  518.98376 518.7007 ]
 [526.88226 526.90704 516.7817  ... 518.49054 531.73315 520.24396]
 [499.21796 507.82153 487.6658  ... 502.55704 498.85074 505.20505]
 ...
 [507.9978  511.47415 505.78442 ... 511.31113 515.17816 508.06317]
 [509.79404 511.2913  494.37088 ... 496.7069  512.6479  506.48886]
 [508.98257 510.27228 501.07715 ... 499.69965 516.30585 495.35336]]
Output: [[512.76294 516.0406  504.8803  ... 512.818   518.98364 518.7012 ]
 [526.8825  526.90704 516.78156 ... 518.4908  531.73315 520.24414]
 [499.21808 507.82123 487.66632 ... 502.55734 498.85034 505.2049 ]
 ...
 [507.99817 511.474   505.78406 ... 511.31113 515.1784  508.06326]
 [509.79324 511.29083 494.37103 ... 496.70755 512.6481  506.48914]
 [508.98206 510.2725  501.07706 ... 499.69983 516.305   495.35355]]
1DConv TVM: 26.067583 ms


In [24]:
print(tvm.lower(s, [A, W, B], simple_mode=True))

# from tvm.script import ir as I
# from tvm.script import tir as T

@I.ir_module
class Module:
    @T.prim_func
    def main(A: T.Buffer((1024, 2048), "float32"), B: T.Buffer((2048, 512), "float32"), C: T.Buffer((1024, 512), "float32")):
        T.func_attr({"from_legacy_te_schedule": T.bool(True), "global_symbol": "main", "tir.noalias": T.bool(True)})
        blockIdx_x = T.launch_thread("blockIdx.x", 128)
        blockIdx_y = T.launch_thread("blockIdx.y", 64)
        threadIdx_x = T.launch_thread("threadIdx.x", 8)
        threadIdx_y = T.launch_thread("threadIdx.y", 8)
        C_1 = T.Buffer((524288,), data=C.data)
        C_1[blockIdx_x * 4096 + threadIdx_x * 512 + blockIdx_y * 8 + threadIdx_y] = T.float32(0)
        for k_outer, k_inner in T.grid(512, 4):
            A_1 = T.Buffer((2097152,), data=A.data)
            B_1 = T.Buffer((1048576,), data=B.data)
            C_1[blockIdx_x * 4096 + threadIdx_x * 512 + blockIdx_y * 8 + threadIdx_y] = C_1[blockIdx_x * 4096 + threadIdx_x * 

In [25]:
%cd {PROJECT_ROOT}
!python -m pytest tests/test_gemm_gpu.py

/content/gdrive/MyDrive/ece5545/a3-WDaugherty
============================= test session starts ==============================
platform linux -- Python 3.9.16, pytest-7.2.2, pluggy-1.0.0
rootdir: /content/gdrive/MyDrive/ece5545/a3-WDaugherty
collected 20 items                                                             

tests/test_gemm_gpu.py ....................                              [100%]

============================= 20 passed in 15.40s ==============================
